In [14]:
import numpy as np
import json_tricks

answer = {}


inputs = json_tricks.load('inputs/inputs.json')



# Matrix Product II

$$A_{5 \times 10} \  B_{10 \times 2}\ C_{2 \times 30}\ D_{30 \times 3}\ E_{3 \times 9}$$

1. What will be the shape of the resulting matrix?
2. How many multiplication of numbers are required at best?

In [15]:
dims = [5,10,2,30,3,9]

def matrix_chain_order(dims):
    n = len(dims) - 1
    m = np.zeros((n,n), dtype=int)
    for l in range(2, n+1):
        for i in range(n-l+1):
            j = i + l -1
            m[i,j] = np.iinfo(np.int32).max
            for k in range(i,j):
                q = m[i,k] + m[k+1,j] + dims[i]*dims[k+1]*dims[j+1]
                if q < m[i,j]:
                    m[i,j] = q
    return m[0,n-1]

min_mult = matrix_chain_order(dims)

answer['task1'] = {
    '1': (dims[0], dims[-1]),  # (5, 9)
    '2': int(min_mult)
}




# Numpy expression

Using Numpy, write a function that calculates the 
following expression:

$$\exp(A^T(B + 2C) + 3I) \mathbf x,$$

where $I$ is an identity matrix of the necessary shape.

In [16]:
def numpy_expression(A, B, C, x):
    # Calculate: exp(A^T (B + 2C) + 3I) x
    # I - identity matrix of necessary shape (square matrix same shape as A^T(B+2C))
    I = np.eye(A.shape[1])  # since A^T shape: (columns of A, rows of A)
    mat = A.T @ (B + 2*C) + 3 * I
    res = np.exp(mat) @ x
    return res

In [17]:
answer['task2'] = []
for one_input in inputs['task2']:
    answer['task2'].append(numpy_expression(**one_input))

# Einstein's Rule

In *Tensor Algebra*, a direct generalization of the Linear Algebra to the case of $N$-dimentional tables called *tensors* (normal matrix), the Einstein's rule exists.

It works as follows: if you see a duplicating upper and lower index in the formula, that means, this index convolves.

For example, the following tensor expression, summation and matrix product are equivalent:

$$a_k^l b_l^m = \sum_{l=1}^L a_k^l b_l^m = AB$$

In this notation subscript means row index and superscript means column index.

<details>
<summary> Note </summary>

> [!NOTE]
> Also at some point it will be important to know that:
> * lower index represents a contravariant dimension of a
> tensor
> * upper index represents a covariant dimension 
> of tensor. But let us omit this part for now.

</details>

# Task

Calculate the following expression written using Einstein's 
rule:

$$a_k^m b_m^n c_n^o d_l^k$$

In [18]:
def einsteins_rule(A, B, C, D):
    # Используем np.einsum для сокращения индексов
    # a_k^m -> 'km', b_m^n -> 'mn', c_n^o -> 'no', d_l^k -> 'lk'
    # Выражение: a_k^m b_m^n c_n^o d_l^k
    # Итог: sum по m,n,k -> индексы l,o остаются
    res = np.einsum('km,mn,no,lk->lo', A, B, C, D)
    return res

In [19]:
answer['task3'] = []
for one_input in inputs['task3']:
    answer['task3'].append(einsteins_rule(**one_input))

# Diagonal Matrix Product

You are given two square matrices: $A$ and $D$, where $A$ is a 
full matrix and $D$ is a diagonal matrix:

$$
A = \begin{bmatrix}
- & \mathbf a_1 & - \\
& \vdots & \\
- & \mathbf a_N & - \\
\end{bmatrix}
$$

$$
D = \textrm{diag}(d_1, d_2, \dots, d_N) = \begin{bmatrix}
d_1 & & & & \\
& d_2 & & & \\
& & d_3 & & \\
& & & \ddots & \\
& & & & d_N 
\end{bmatrix}
$$

Write a program to calculate the result of $DA$ and $AD$ in 
the fastest possible way.

In [ ]:
def diag_prod_DA(A, D):
    # Handle both cases: D as 1D array or 2D diagonal matrix
    if D.ndim == 1:
        # D is 1D array representing diagonal elements
        return D[:, None] * A
    else:
        # D is 2D diagonal matrix
        return D @ A

def diag_prod_AD(A, D):
    # Handle both cases: D as 1D array or 2D diagonal matrix
    if D.ndim == 1:
        # D is 1D array representing diagonal elements
        return A * D
    else:
        # D is 2D diagonal matrix
        return A @ D

answer['task4_1'] = []
answer['task4_2'] = []

NameError: name 'answer' is not defined

In [ ]:
for one_input in inputs['task4']:
    answer['task4_1'].append(diag_prod_DA(**one_input))
    answer['task4_2'].append(diag_prod_AD(**one_input))

# Sparse Matrix Product

You are given two matrices of the same shape: $A$ and $B$. Matrix $A$ is full
and is given in the form of `numpy.ndarray`.

The second matrix $B$ is **sparse**. That means that the 
majority of the items are equal to $0$ except for $M$. This matrix is given
as a set of non-zero elements of this matrix in form of $3 \times M$ `numpy.ndarray` as row-column-value tuple (COO sparse matrix form):

$$
\begin{bmatrix}
r_1 & c_1 & v_1 \\
r_2 & c_2 & v_2 \\
& \vdots & \\
r_M & c_M & v_M \\
\end{bmatrix}
$$

If in this struct two items correspond to the same location, consider the latter is correct.

Write the most efficient program that calculates $AB$.

Also return the ratio between the number of multiplication operations that are needed to calculate the sparse product and the number of operations for full product.

In [ ]:
def sparse_matrix_product(A, B_sparse):
    rows = B_sparse[0]
    cols = B_sparse[1]
    vals = B_sparse[2]

    m, n = A.shape
    M = len(rows)

    # Determine the number of columns in B
    if len(cols) > 0 and len(cols) == len(rows) and len(rows) > 0:
        n_cols_B = int(np.max(cols)) + 1
    else:
        n_cols_B = 0

    res = np.zeros((m, n_cols_B))

    # Process each non-zero element
    # The problem statement says: "If in this struct two items correspond to the same location, consider the latter is correct."
    # This means we should process in order and overwrite duplicates
    processed_positions = {}
    for i in range(M):
        r = int(rows[i])
        c = int(cols[i])
        v = vals[i]
        key = (r, c)
        processed_positions[key] = v  # This will overwrite previous values
    
    # Now compute the matrix multiplication
    for (r, c), v in processed_positions.items():
        if 0 <= r < n and 0 <= c < n_cols_B:  # bounds checking
            res[:, c] += v * A[:, r]

    # Calculate operation counts
    full_mult = m * n * n_cols_B if n_cols_B > 0 else 0
    sparse_mult = len(processed_positions) * m if len(processed_positions) > 0 else 0

    ratio = sparse_mult / full_mult if full_mult > 0 else 0.0

    return res, float(ratio)


In [ ]:
answer['task5'] = []
for one_input in inputs['task5']:
    result = sparse_matrix_product(**one_input)
    answer['task5'].append(result)

In [25]:
json_tricks.dump(answer, '.answer.json')

'{"task1": {"1": [5, 9], "2": 424}, "task2": [{"__ndarray__": [[129.28183698947615], [1.2706705664732254]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-19.814866356714443], [-22026.45231891272]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-1.0202138409972563], [-196988010.4959191]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[148.4198970495757], [7.389391561558553]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[970330390.7646337], [-65970.20108435386]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-445.23947730772977], [-3.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-23502.394195740686], [-0.005612492598890043]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-0.002478721716706869], [-2.9469672762675123e-07]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[5.43655136